# 📉 Capítulo 4 – Detección de Anomalías con VARIMA

En este capítulo se estudia el modelo **VARIMA (Vector AutoRegressive Integrated Moving Average)**, la generalización multivariante de ARIMA. Dado que en nuestro caso trabajamos con **señales multivariantes** donde las relaciones entre variables son esenciales (correlaciones cruzadas, desfases, efectos de acoplamiento), **ARIMA univariado no es suficiente**. VARIMA proporciona un marco estadístico para **modelar conjuntamente** varias series y detectar anomalías como **innovaciones inusuales** respecto al comportamiento esperado del sistema.

---

## ❌ ¿Por qué **no** ARIMA y sí VARIMA?

- **ARIMA es univariado**: modela una única serie \(y_t\). Incluso con ARIMAX/SARIMAX (variables exógenas), el objetivo sigue siendo una **sola** variable dependiente. Esto impide capturar **anomalías relacionales** (p. ej., patrones anómalos que emergen solo por la combinación de dos o más señales).
- **Agregación de ARIMAs por canal**: ajustar un ARIMA por cada variable y luego combinar residuos **ignora la covarianza** entre series. Se pierden anomalías que solo son evidentes en el **espacio conjunto** (p. ej., temperatura normal + presión normal pero **combinación** improbable).
- **VARIMA modela el vector completo** \(\mathbf{y}_t \in \mathbb{R}^k\): incorpora **dependencias cruzadas** (cómo la variable \(i\) hoy depende de la variable \(j\) ayer), dando lugar a **innovaciones multivariantes** cuya magnitud (p. ej., distancia de Mahalanobis) permite una **detección coherente** a nivel de sistema.

> En síntesis: con datos multivariantes y sin supuestos claros, **VARIMA** (o su forma práctica **VARMA/VARMAX**) es el baseline estadístico adecuado.

---

## 🔍 Intuición

VARIMA asume que el vector de observaciones \(\mathbf{y}_t\) puede describirse mediante:
1) **Dependencias autoregresivas** entre componentes y rezagos (\(\text{VAR}\)).  
2) **Medias móviles** de innovaciones pasadas (\(\text{VMA}\)).  
3) **Diferenciación** para lograr estacionariedad (la “I” de VARIMA).

Bajo ese modelo, lo “esperado” para el sistema es \(\hat{\mathbf{y}}_t\). Una **anomalía** ocurre cuando la **innovación** \(\boldsymbol{\varepsilon}_t = \mathbf{y}_t - \hat{\mathbf{y}}_t\) es **improbable** según la covarianza aprendida en entrenamiento.

---

## 🧠 Fundamento teórico (resumen)

Sea \(\mathbf{y}_t \in \mathbb{R}^k\). Un VARIMA\((p,d,q)\) puede escribirse, con el operador de retardo \(B\), como:

\[
\Phi(B)\,(1-B)^d\,\mathbf{y}_t \;=\; \Theta(B)\,\boldsymbol{\varepsilon}_t,\quad 
\boldsymbol{\varepsilon}_t \sim \mathcal{WN}_k(\mathbf{0},\,\Sigma),
\]

donde:
- \(\Phi(B) = I - \Phi_1 B - \cdots - \Phi_p B^p\) (parte **VAR**),
- \(\Theta(B) = I + \Theta_1 B + \cdots + \Theta_q B^q\) (parte **VMA**),
- \(d\) es el orden de **diferenciación** (puede ser común o por componente),
- \(\boldsymbol{\varepsilon}_t\) son **innovaciones** (ruido blanco vectorial con covarianza \(\Sigma\)).

> En práctica, muchas implementaciones operativas usan **VARMA/VARMAX** (donde la “I” se maneja con pre-diferenciación) por disponibilidad de software y estabilidad numérica.

---

## 📐 Detección de anomalías con VARIMA

**Procedimiento**:

1. **Entrenamiento**  
   Ajustar VARIMA/VARMA en el conjunto de entrenamiento \(\{\mathbf{y}_t\}\).  
   Estimar la **covarianza de innovaciones** \(\Sigma\) a partir de los residuos de entrenamiento.

2. **Puntuación (score)**  
   Para cada instante, obtener la innovación \(\boldsymbol{\varepsilon}_t = \mathbf{y}_t - \hat{\mathbf{y}}_t\) (pronóstico one-step-ahead) y calcular el **score**:
   - **Mahalanobis** (recomendado):  
     \[
     D_t^2 \;=\; \boldsymbol{\varepsilon}_t^\top \,\Sigma^{-1}\, \boldsymbol{\varepsilon}_t
     \]
     Bajo supuestos clásicos, \(D_t^2 \sim \chi^2_k\).
   - **L2 estandarizado** (alternativa simple):  
     \( \sum_i (\varepsilon_{t,i}/\hat{\sigma}_i)^2 \) (aprox. \(\chi^2_k\) si independencia).

3. **Umbralización**  
   Dos opciones “clásicas”:
   - **Teórica** (sin etiquetas): \(D_t^2 > \chi^2_{k,\,1-\alpha}\) ⇒ anomalía.  
   - **Por *contamination***: umbral = cuantil \(1-\alpha\) del score en **training** (controla la tasa de candidatos anómalos).

4. **Post-proceso opcional**  
   Persistencia (exigir \(k\) superaciones consecutivas), histeresis (activar con \(\tau_{\text{on}}\), desactivar con \(\tau_{\text{off}}\)), o suavizado de scores para reducir parpadeos.

---

## ⚙️ Parámetros clave y selección

- **Ordenes \(p, q\)**: nº de rezagos VAR/VMA. Se seleccionan con **AIC/BIC** y diagnóstico de residuos.  
- **Diferenciación \(d\)**: pre-procesado para (cuasi) estacionariedad; puede requerir **d** distinto por variable o un **d** común (más simple).  
- **Tratamiento estacional**: si hay estacionalidad clara, usar SARIMA multivariante (VARIMA con términos estacionales) o diferenciar por periodo.  
- **Cointegración**: si existen relaciones de equilibrio a largo plazo entre variables, considerar **VECM** (fuera de alcance aquí, pero relevante).  
- **Covarianza \(\Sigma\)**: estimada en training; una regularización leve (p. ej., \(\lambda I\)) mejora estabilidad numérica.

> En producción, es habitual usar **VARMAX** (statespace) y operar con **pronóstico one-step-ahead** actualizando el filtro con cada observación.

---

## ✅ Ventajas

- **Captura dependencias cruzadas** entre variables y rezagos (anomalías relacionales).  
- Umbralización **probabilística** (p. ej., \(\chi^2_k\)) o por **contamination** (operativa).  
- Marco clásico, **interpretable**: residuos, innovaciones, covarianza.

## 🚫 Limitaciones

- Requiere **más datos** y es más sensible a especificación que ARIMA univariado.  
- Supone **linealidad** y ruido aproximadamente gaussiano (o al menos simétrico).  
- Selección de órdenes y diferenciación puede ser **costosa** y dependiente del dominio.  
- Si las dinámicas son fuertemente no lineales, DL puede superar a VARIMA.

---

## 🧪 Ejemplo ilustrativo (multivariante)

Supón dos señales de un subsistema: **RPM** y **temperatura** del refrigerante.  
Por separado, sus valores pueden parecer “normales”. Sin embargo, un **VARIMA** que captura la **relación normal** entre ambas (p. ej., a RPM bajas suele corresponder una temperatura moderada) detectará como **anómala** una combinación improbable (RPM bajas + temperatura anormalmente alta) porque la **innovación conjunta** \(\boldsymbol{\varepsilon}_t\) tendrá **alta energía** \(D_t^2\) respecto a \(\Sigma\).

---

## 🧭 Flujo práctico (alineado con la validación temporal)

1. **Entrenar** VARIMA/VARMA/VARMAX en *training*.  
2. **Fijar umbral**:  
   - Sin etiquetas ⇒ \(\chi^2_{k,1-\alpha}\).  
   - Con validación ⇒ **contamination** (cuantil) ajustado para FA/h objetivo.  
3. **Evaluar** en *validation* (ajustar hiperparámetros/umbral si procede).  
4. **Reportar** en *test* (sin tocar el umbral).  

---

## 📚 Referencias clásicas

- **Lütkepohl, H.** *New Introduction to Multiple Time Series Analysis*. Springer.  
- **Box, G. E. P., Jenkins, G. M., Reinsel, G. C., Ljung, G. M.** *Time Series Analysis: Forecasting and Control*. Wiley.  
- **Shumway, R. H., Stoffer, D. S.** *Time Series Analysis and Its Applications*. Springer.

